In [ ]:
# Partial Least Squares Discriminant Analysis
# Change the NAME.csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Carregar o arquivo CSV
try:
    df = pd.read_csv('NAME.csv')
except FileNotFoundError:
    print("Erro: Arquivo 'NAME.csv' não encontrado. Certifique-se de que o arquivo está na mesma pasta ou forneça o caminho completo.")
    # Você pode adicionar um código aqui para fazer upload do arquivo, se necessário.
    # from google.colab import files
    # uploaded = files.upload()
    # df = pd.read_csv('NAME.csv')
    exit() # Termina a execução se o arquivo não for encontrado

# Separar os dados: A primeira coluna é a classe, o resto são os espectros
X = df.iloc[:, 1:].values  # Dados dos espectros (todas as colunas, exceto a primeira)
y = df.iloc[:, 0].values   # Classes (primeira coluna)
wavenumbers = df.columns[1:].astype(float) # A primeira linha (coluna 1 em diante) são os números de onda

# Converter as classes para formato numérico usando LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Criar o modelo PLS-DA
# O número de componentes é um hiperparâmetro que pode ser otimizado
n_components = min(X_train.shape[0], X_train.shape[1], 10) # Exemplo: usar até 10 componentes ou o mínimo possível
plsda = PLSRegression(n_components=n_components)

# Treinar o modelo
plsda.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred_raw = plsda.predict(X_test)

# Para classificação, PLS-DA geralmente requer um passo adicional para converter as saídas contínuas em classes.
# Uma abordagem comum é usar um classificador nos scores latentes ou binarizar as previsões.
# Uma maneira simples para múltiplas classes é atribuir a classe com a maior pontuação (assumindo que o LabelEncoder mapeou as classes sequencialmente).
# Uma abordagem mais robusta seria usar um classificador (como Logistic Regression ou SVM) nos scores latentes.

# Abordagem simples: arredondar e clipar para os limites das classes codificadas
y_pred_rounded = np.round(y_pred_raw).astype(int)
y_pred = np.clip(y_pred_rounded, 0, len(label_encoder.classes_) - 1) # Garante que as previsões estejam dentro dos limites das classes codificadas

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia do modelo PLS-DA: {accuracy:.2f}')

print('\nRelatório de Classificação:')
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Visualizar os resultados do PLS-DA (opcional)
# Scores latentes
X_train_r, Y_train_r = plsda.transform(X_train, y_train)
X_test_r, Y_test_r = plsda.transform(X_test, y_test)

plt.figure(figsize=(10, 8))
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k'] # Cores para as classes

for i, target_name in enumerate(label_encoder.classes_):
    plt.scatter(X_test_r[y_test == i, 0], X_test_r[y_test == i, 1], color=colors[i % len(colors)], label=target_name)

plt.xlabel(f'Componente Latente 1 ({plsda.x_scores_[:, 0].var():.2f}% variância explicada)')
plt.ylabel(f'Componente Latente 2 ({plsda.x_scores_[:, 1].var():.2f}% variância explicada)')
plt.title('PLS-DA: Scores Latentes dos Dados de Teste')
plt.legend()
plt.grid(True)
plt.show()

# Coeficientes de regressão do PLS-DA (mostra a importância das variáveis - números de onda)
# Note que em PLS-DA, a interpretação dos coeficientes pode ser complexa.
# Muitas vezes, os "loadings" ou "VIP scores" (Variable Importance in Projection) são usados para identificar variáveis importantes.
# O sklearn PLSRegression não calcula VIP scores diretamente. Podemos usar os loadings.

# Loadings X
# loadings_x = plsda.x_loadings_

# Coeficientes de regressão (relação entre X e Y latentes)
# regression_coefficients = plsda.coef_

# Se você quiser plotar algo que se relacione com a importância dos números de onda,
# pode plotar os loadings do primeiro componente latente, por exemplo.
# Loadings altos (positivos ou negativos) neste componente indicam que este número de onda contribui significativamente para essa componente.

if plsda.x_loadings_.shape[1] > 0:
    plt.figure(figsize=(12, 6))
    plt.plot(wavenumbers, plsda.x_loadings_[:, 0])
    plt.xlabel('Número de Onda (cm⁻¹)')
    plt.ylabel('Loading do Componente 1')
    plt.title('Loadings do Primeiro Componente PLS-DA')
    plt.gca().invert_xaxis() # Números de onda geralmente são plotados em ordem decrescente
    plt.grid(True)
    plt.show()
else:
    print("Não foi possível calcular/plotar os loadings, verifique o número de componentes.")



In [ ]:
# PLS-DA with confidence elipse, english.

import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`.

    Parameters
    ----------
    x, y : array_like, shape (n, ).
        Input data.
    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.
    n_std : float
        The number of standard deviations to determine the ellipse's size.
    facecolor : str
        Color of the ellipse face. Defaults to 'none'.
    kwargs : dict
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of a symmetric
    # positive definite matrix in two dimensions
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the stdandard deviation of x from the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # Calculating the stdandard deviation of y from the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D().rotate_deg(45).scale(scale_x, scale_y).translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


# Calculate latent scores for the full dataset (train + test) for better ellipse visualization
X_full_r, Y_full_r = plsda.transform(X, y_encoded)

plt.figure(figsize=(10, 8))
colors = ['r', 'g', 'b', 'c', 'm', 'y', 'k'] # Colors for the classes

ax = plt.gca() # Get current axes

for i, target_name in enumerate(label_encoder.classes_):
    # Plot the points
    ax.scatter(X_full_r[y_encoded == i, 0], X_full_r[y_encoded == i, 1],
               color=colors[i % len(colors)], label=target_name, alpha=0.6)

    # Plot the confidence ellipse
    confidence_ellipse(X_full_r[y_encoded == i, 0], X_full_r[y_encoded == i, 1], ax,
                       n_std=2.0, edgecolor=colors[i % len(colors)], linestyle='--', alpha=0.5) # 2 std for ~95% confidence

plt.xlabel(f'Latent Component 1 (Explained Variance: {plsda.x_scores_[:, 0].var():.2f}%)')
plt.ylabel(f'Latent Component 2 (Explained Variance: {plsda.x_scores_[:, 1].var():.2f}%)')
plt.title('PLS-DA: Latent Scores with Confidence Ellipses')
plt.legend()
plt.grid(True)
plt.show()